In [ ]:
import pandas as pd
import numpy as np

import argparse

from data_module_F import *
from model_module_F import *
from feature_module_F import *
from evaluation_module_F import *


#from Resnet1D_builder import *
from EfficientNet1D_builder import *
from model_builder import *

import sys, os

tf.random.set_seed(155)

In [ ]:
path = "/net/home/dmederer/Transferability_efficient/processed_original_data.csv" ## data path
exp = "comparison_training/" ## experiment name 
route = '/net/home/dmederer/Transferability_efficient/'## store experiment's files name
epochs = 300 ## num epoch
lr = 0.0005 ### learning rate
kind = 'efficientnet' ## type of model to train

epochs = 300
lr = 0.0005
kind = 'efficientnet'

In [ ]:
######## GPU RAM memory ##########
os.environ["CUDA_VISIBLE_DEVICES"]="1"

gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 15*1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=1024*35)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
            print(e)

In [ ]:
## CHOOSE the right file path

path_g = route + '{}_spatialCV_real_only/'.format(exp)
#path_g = route + '{}_spatialCV_rtm_only/'.format(exp)
#path_g = route + '{}_spatialCV_low_var_test/'.format(exp)
#path_g = route + '{}_spatialCV_frequency/'.format(exp)


create_path(path_g)

In [ ]:
os.chdir("/net/home/dmederer/Transferability_efficient/")

In [ ]:
#### Load processed original data 
db, X, y = read_db(path ,sp=True)

# Load processed rtm data
db_rtm, X_rtm, y_rtm = read_db("processed_rtm_data.csv" ,sp=True)

In [ ]:
db

In [ ]:
db_rtm

In [ ]:
for gp in range(1, 43):
    #if gp == 41: # to exclude certain datasets from training
     #   continue
    #### Split training and test data
    db_test = db.groupby("dataset").get_group(gp)
    val_x, val_y, samp_w_val = data_prep('400', db_test, Traits, w_train = db_test.loc[:,:'Site'], multi= True)

    db_train = db.drop(db_test.index, axis=0)

    fgp = balanceData(db_train.reset_index(drop=True).loc[:,'Al content (mg/cm²)':], 
                      db_train.loc[:,:'Site'].reset_index(drop=True), Traits, random_state=300)
    
    ### REMOVE THIS WHEN TRAINING ONLY REAL DATA
    
    ### Add a random subset of rtm data equal to the training fold size
    #num_rows = len(fgp)
    #db_rtm_subset = db_rtm.sample(n=num_rows, random_state=42)
    
    # Concatenate original data and rtm subset
    #fgp = pd.concat([fgp, db_rtm_subset], axis=0, sort=False).reset_index(drop=True)
    
    # OPTIONAL: only use rtm data for training (rtm-only run)
    #fgp = db_rtm_subset 


    ###
    
    train_x, train_y, samp_w_tr = data_prep('400', fgp, Traits, w_train = fgp.loc[:,:'Site'], multi= True)

    scaler_list = save_scaler(train_y, save=True, dir_n = path_g, k = 'db{}'.format(gp))

# LEAVE OUT FOR RTM DATA ONLY (keep train_ds and test_ds)
    if(samp_w_tr is not None):
        if (samp_w_tr.sum().sum() !=0):    
            train_ds = dataset(train_x, train_y, pd.DataFrame(samp_w_tr), scaler_list, 
                               Traits, shuffle=True,augment=False)
            test_ds = dataset(val_x, val_y, samp_w_val, scaler_list, Traits)
    else:
        train_ds = dataset(train_x, train_y, None, scaler_list, Traits, shuffle=True,augment=False) #!!!!!!
        test_ds = dataset(val_x, val_y, None, scaler_list, Traits)

    ##### Model definition  and training #######
    input_shape = train_x.shape[1]
    output_shape = train_y.shape[1]


    EPOCHS = epochs 
    best_model = model_definition(input_shape, output_shape,lr = lr, kind= kind)

    checkpoint_path = path_g + 'checkpoint_db{}'.format(gp)
    create_path(checkpoint_path)

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path + "/epoch{epoch:02d}-val_root_mean_squared_error{val_root_mean_squared_error:.2f}.hdf5",
    save_weights_only=True,
    monitor = 'val_root_mean_squared_error',
    mode='min',
    save_best_only=True)

    his = best_model.fit(train_ds,
                    validation_data = test_ds,
                    epochs = EPOCHS,
                    verbose=1, callbacks = [model_checkpoint_callback])


    val_acc_per_epoch = his.history['val_root_mean_squared_error']
    best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1    

    path_trial = path_g + "Model_db{}.json".format(gp)
    path_best = checkpoint_path + "/epoch{0:02d}-val_root_mean_squared_error{1:.2f}.hdf5".format(best_epoch,min(val_acc_per_epoch))
    path_w = path_g + 'Trial_db{}_weights.h5'.format(gp)
    save_model(best_model, path_trial, path_best, path_w)

    pred = scaler_list.inverse_transform(best_model.predict(val_x))
    pred_df = pd.DataFrame(pred, columns = val_y.columns+ ' Predictions')
    pred_df.to_csv(path_g + 'Predictions_db{}.csv'.format(gp))

    obs_pf = pd.DataFrame(val_y)
    obs_pf.to_csv(path_g + 'Observations_db{}.csv'.format(gp))

    test = all_scores(Traits,Traits,obs_pf, pred_df,None)
    test.to_csv(path_g + 'Global_all_db{}.csv'.format(gp))

In [ ]:
### FINAL STEP AFTER TRAINING ALL FOLDS ###

In [ ]:
pwd

In [ ]:
# Set the directory path
directory_path = '/net/home/dmederer/Transferability_efficient/comparison_training/_spatialCV_real_only/'
#directory_path = '/net/home/dmederer/Transferability_efficient/comparison_training/_spatialCV_real_only_test/'
#directory_path = '/net/home/dmederer/Transferability_efficient/comparison_training/_spatialCV_rtm_only/'
#directory_path = '/net/home/dmederer/Transferability_efficient/comparison_training/_spatialCV_low_var/'
#directory_path = '/net/home/dmederer/Transferability_efficient/comparison_training/_spatialCV_low_var_test/'
#directory_path = '/net/home/dmederer/Transferability_efficient/comparison_training/_spatialCV_frequency/'

# Change the current working directory
os.chdir(directory_path)

In [ ]:
# Re-define path_g to plot directory
#path_g = /net/home/dmederer/Transferability_efficient/plot_folder

path_g = route + 'plot_folder/'


create_path(path_g)

In [ ]:
# Load and concatenate observations

# Initialize an empty list to store the dataframes
observation_dataframes = []

# Loop through the numbered filenames
for i in range(1, 43):
    filename = f"Observations_db{i}.csv"  # Assuming the files have a ".csv" extension
    
    try:
        df = read_db(filename)  # Load the dataframe from the file
        observation_dataframes.append(df)  # Append the dataframe to the list
    except FileNotFoundError:
        print(f"File {filename} not found. Skipping...")

# Concatenate the dataframes
observation_combined_df = pd.concat(observation_dataframes, ignore_index=True)

# Save the concatenated dataframe
observation_combined_df.to_csv(path_g + "Observations_combined_real_only.csv", index=False)
#observation_combined_df.to_csv(path_g + "Observations_combined_real_only_test.csv", index=False)
#observation_combined_df.to_csv(path_g + "Observations_combined_rtm_only.csv", index=False)
#observation_combined_df.to_csv(path_g + "Observations_combined_low_var.csv", index=False)
#observation_combined_df.to_csv(path_g + "Observations_combined_low_var_test_nodecay.csv", index=False)
#observation_combined_df.to_csv(path_g + "Observations_combined_frequency.csv", index=False)


In [ ]:
observation_combined_df

In [ ]:
# Load and concatenate predictions

# Initialize an empty list to store the dataframes
prediction_dataframes = []

# Loop through the numbered filenames
for i in range(1, 43):
    filename = f"Predictions_db{i}.csv"  # Assuming the files have a ".csv" extension
    
    try:
        df = read_db(filename)  # Load the dataframe from the file
        prediction_dataframes.append(df)  # Append the dataframe to the list
    except FileNotFoundError:
        print(f"File {filename} not found. Skipping...")

# Concatenate the dataframes
prediction_combined_df = pd.concat(prediction_dataframes, ignore_index=True)

# Save the concatenated dataframe
prediction_combined_df.to_csv(path_g + "Predictions_combined_real_only.csv", index=False)
#prediction_combined_df.to_csv(path_g + "Predictions_combined_real_only_test.csv", index=False)
#prediction_combined_df.to_csv(path_g + "Predictions_combined_rtm_only.csv", index=False)
#prediction_combined_df.to_csv(path_g + "Predictions_combined_low_var.csv", index=False)
#prediction_combined_df.to_csv(path_g + "Predictions_combined_low_var_test_nodecay.csv", index=False)
#prediction_combined_df.to_csv(path_g + "Predictions_combined_frequency.csv", index=False)


In [ ]:
prediction_combined_df

In [ ]:
observation_combined_df.sort_values(by=["LMA (g/m²)"], ascending=False).head(10)

In [ ]:

test = all_scores(Traits,Traits,observation_combined_df, prediction_combined_df,None)
test.to_csv(path_g + 'Evaluation_transferability_combined_real_only.csv')
#test.to_csv(path_g + 'Evaluation_transferability_combined_real_only_test.csv')
#test.to_csv(path_g + 'Evaluation_transferability_combined_with_rtm_only.csv')
#test.to_csv(path_g + 'Evaluation_transferability_combined_with_low_var.csv')
#test.to_csv(path_g + 'Evaluation_transferability_combined_with_low_var_test_nodecay.csv')
#test.to_csv(path_g + 'Evaluation_transferability_combined_with_frequency.csv')

In [ ]:
test